In [ ]:
# some_file.py
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '../src')

import model

llm = model.llm_1

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

gapi_key = os.getenv("GOOGLE_API_KEY", None)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
g_llm = ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True)

In [ ]:
from langchain.schema.messages import HumanMessage, SystemMessage

chat_llm = g_llm

In [ ]:
instructions = SystemMessage(content="""
You are a surfer dude, having a conversation about the surf conditions on the beach.
Respond using surfer slang.
""")

In [ ]:
question = HumanMessage(content="What is the weather like?")

In [ ]:
response = chat_llm([
    instructions,
    question
] )

print(response)

In [ ]:
from langchain.chat_models.openai import ChatOpenAI
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import LLMChain

chat_llm = g_llm

prompt = PromptTemplate(template="""You are a surfer dude, having a conversation about the surf conditions on the beach.
Respond using surfer slang.

Context: {context}
Question: {question}
""", input_variables=["context", "question"])

chat_chain = LLMChain(llm=chat_llm, prompt=prompt)

current_weather = """
    {
        "surf": [
            {"beach": "Fistral", "conditions": "6ft waves and offshore winds"},
            {"beach": "Polzeath", "conditions": "Flat and calm"},
            {"beach": "Watergate Bay", "conditions": "3ft waves and onshore winds"}
        ]
    }"""

response = chat_chain.run(
    context=current_weather,
    question="What is the weather like on Watergate Bay?"
)

print(response)

# Giving a Chat Model Memory

In [ ]:
prompt = PromptTemplate(template="""You are a surfer dude, having a conversation about the surf conditions on the beach.
Respond using surfer slang.

Chat History: {chat_history}
Context: {context}
Question: {question}
""", input_variables=["chat_history", "context", "question"])

In [ ]:
from langchain.chains.conversation.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", input_key="question", return_messages=True)

chat_chain = LLMChain(llm=chat_llm, prompt=prompt, memory=memory)

In [ ]:
response = chat_chain.run(context=current_weather, question="Hi, I am at Watergate Bay. What is the surf like?")
print(response)
print()

response = chat_chain.run(context=current_weather, question="Where I am?")
print(response)
print()
print(len(str(memory)))